In [1]:
import os
import time
from glob import glob

import torch
from torch.utils.data import DataLoader
import torch.nn as nn

from data import DriveDataset
from model import build_unet
from loss import DiceLoss, DiceBCELoss
from utils import seeding, create_dir, epoch_time

In [2]:
def train(model, loader, optimizer, loss_fn, device):
    epoch_loss =0.0
    model.train()
    for x, y in loader:
        x = x.to(device, dtype = torch.float32)
        y = y.to(device, dtype = torch.float32)
        optimizer.zero_grad()
        y_pred = model(x)
        
        loss = loss_fn(y_pred, y)
        
        
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    epoch_loss = epoch_loss/len(loader)
    return epoch_loss

In [3]:
if __name__ == "__main__":
    seeding(42)
    
    create_dir("files")
    
    train_x = glob("C:/Users/subha/OneDrive/Documents/th_cancer_data/new_image/train/image/*")
    train_y = glob("C:/Users/subha/OneDrive/Documents/th_cancer_data/new_image/train/mask/*")
    
    
    #Hyperparameter
    size = (256,128)
    batch_size = 4
    num_epochs = 70
    lr = 1e-4
    checkpoint_path = "files/checkpoint.pth"
    
    train_dataset = DriveDataset(train_x, train_y)
    
    train_loader = DataLoader(
        dataset= train_dataset,
        batch_size = batch_size,
        shuffle = True,
        num_workers = 2
    )
    device = torch.device("cuda")
    model = build_unet()
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,"min",patience=5,verbose=True)
    loss_fn = DiceBCELoss()
    training_losses = []
    for epoch in range(num_epochs):
        start_time = time.time()
        
        train_loss = train(model, train_loader, optimizer, loss_fn, device)
        print(train_loss)
        training_losses.append(train_loss)

1.285780017192547
1.1103428556368902
1.0498978449748113
1.0438353969500616
0.9184735692464389
0.9316230507997366
0.8879657066785372
0.8663774499526391
0.8986446811602666
0.8717420880611126
0.8234169116387
0.7695363943393414
0.7594384184250464
0.7600177984971267
0.6255777455293215
0.5936291240728818
0.5608492493629456
0.6022255466534541
0.5233489825175359
0.47081934259488034
0.47225688512508684
0.46619802254896897
0.460735185788228
0.4194659957518944
0.41567269426125747
0.4494861295590034
0.36794994198358977
0.39080358010071975
0.3721074301462907
0.39443698754677403
0.36302905357800996
0.40151020655265224
0.3751687957690312
0.339484283557305
0.3354173657985834
0.28066532428448016
0.2943683553200502
0.2548070229016818
0.26256082035028017
0.3314824218933399
0.27414755752453435
0.272367903819451
0.2982219411776616
0.26299065122237575
0.2855097789030809
0.2724028607973686
0.2385450555728032
0.19900254675975212
0.20690267246503097
0.24023983340996963
0.20326087451898134
0.21938793131938347
0

In [4]:
img = "C:\\Users\\subha\\OneDrive\\Documents\\Thyroid Ultrasound Images\\90_1.jpg"
mask = "C:\\Users\\subha\\OneDrive\\Documents\\Thyroid Ultrasound Images\\90_mask1.jpg"

In [5]:
import cv2
import numpy as np
image = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
image = image[58:250,66:450]
image = image/255.0
image = np.expand_dims(image,axis = 0)
image = np.expand_dims(image,axis = 0)
image = image.astype(np.float32)
image = torch.from_numpy(image)
        
mask = cv2.imread(mask,cv2.IMREAD_GRAYSCALE)
mask = mask[58:250,66:450]         
mask = mask/255.0
mask = np.expand_dims(mask,axis = 0)
mask = mask.astype(np.float32)
mask = torch.from_numpy(mask)


image = image.to(device, dtype = torch.float32)
mask = mask.to(device, dtype = torch.float32)

In [6]:
image.shape

torch.Size([1, 1, 192, 384])

In [7]:
torch.save(model.state_dict(), checkpoint_path)

In [8]:
model.eval()
output = model(image)

In [9]:
loss = loss_fn(output, mask)
loss

tensor(1.2245, device='cuda:0', grad_fn=<AddBackward0>)

In [10]:
output = output*255
output = output.to("cpu", dtype = torch.float32)
output = output.detach().numpy()
output = np.uint8(output[0][0])

In [11]:
output

array([[ 89, 177, 151, ..., 193, 180, 118],
       [ 75, 162, 148, ...,   5,  20, 194],
       [ 98, 191, 196, ...,  52, 254, 104],
       ...,
       [ 94, 180, 193, ..., 238, 168, 135],
       [102, 163, 161, ...,   8, 172, 122],
       [ 47, 100,  86, ..., 130, 104, 101]], dtype=uint8)

In [12]:
#cv2.imshow(output)
from PIL import Image
img = Image.fromarray(output,mode='L')
img.save('train_pic_MK6.png')

In [13]:
import pandas as pd
d = pd.DataFrame(training_losses)
d

,0
0,1.285780
1,1.110343
2,1.049898
3,1.043835
4,0.918474
...,...
65,0.197652
66,0.217212
67,0.185189
68,0.243641


In [14]:
d.to_csv ('train_MK6.csv')

In [17]:
from torchsummary import summary

In [18]:
summary(model, (1, 256,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 128]             640
       BatchNorm2d-2         [-1, 64, 256, 128]             128
              ReLU-3         [-1, 64, 256, 128]               0
            Conv2d-4         [-1, 64, 256, 128]             640
       BatchNorm2d-5         [-1, 64, 256, 128]             128
              ReLU-6         [-1, 64, 256, 128]               0
        conv_block-7         [-1, 64, 256, 128]               0
         MaxPool2d-8          [-1, 64, 128, 64]               0
     encoder_block-9  [[-1, 64, 256, 128], [-1, 64, 128, 64]]               0
           Conv2d-10         [-1, 128, 128, 64]          73,856
      BatchNorm2d-11         [-1, 128, 128, 64]             256
             ReLU-12         [-1, 128, 128, 64]               0
           Conv2d-13         [-1, 128, 128, 64]          73,856
      BatchNorm2d-14     